In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ta
import optuna

sns.set_theme()

C:\Users\arant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataclasses import dataclass

@dataclass
class Operation:
    time: str
    price: float
    stop_loss: float
    take_profit: float
    n_shares: int
    type: str

In [3]:
def get_portfolio_value(cash: float, long_ops: list[Operation], short_ops: list[Operation], current_price: float, n_shares: int) -> float:
    val = cash

    for long_position in long_ops:
        long_pl = current_price * long_position.n_shares * (1 - COM)
        val += long_pl

    for short_position in short_ops:
        short_pl = (short_op.price - current_price) * short_position.n_shares * (1 - COM)
        val += short_pl

    return val

In [ ]:
# MODIFICACIÓN DE DF

data = pd.read_csv('Binance_BTCUSDT_1h.csv', skiprows=1).dropna()
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data

C:\Users\arant\AppData\Local\Temp\ipykernel_47924\2245535018.py:4: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data['Date'] = pd.to_datetime(data['Date'], errors='coerce', dayfirst=True)


,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,1758582000000,2025-09-22 23:00:00,BTCUSDT,112643.25,112739.14,112592.20,112650.99,135.310950,1.524494e+07,34083
1,1758578400000,2025-09-22 22:00:00,BTCUSDT,112969.99,112970.00,112594.33,112643.25,289.607150,3.264691e+07,42836
2,1758574800000,2025-09-22 21:00:00,BTCUSDT,112781.87,112970.00,112602.79,112969.99,293.311560,3.307493e+07,42931
3,1758571200000,2025-09-22 20:00:00,BTCUSDT,112122.90,112977.41,111975.28,112781.88,596.840050,6.707508e+07,93553
4,1758567600000,2025-09-22 19:00:00,BTCUSDT,112429.12,112600.87,111936.40,112122.90,1307.373650,1.467768e+08,126232
...,...,...,...,...,...,...,...,...,...,...
70822,1502956800000,2017-08-17 08:00:00,BTCUSDT,4333.32,4377.85,4333.32,4360.69,0.972807,4.239504e+03,28
70823,1502953200000,2017-08-17 07:00:00,BTCUSDT,4316.62,4349.99,4287.41,4349.99,4.443249,1.924106e+04,25
70824,1502949600000,2017-08-17 06:00:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,3.128231e+04,36
70825,1502946000000,2017-08-17 05:00:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,1.003048e+05,102


In [7]:
def rsi(data: pd.DataFrame, rsi_window: int, rsi_lower: int, rsi_upper: int) -> tuple:
    rsi_indicator = ta.momentum.RSIIndicator(data['Close'], window=rsi_window)
    rsi = rsi_indicator.rsi()

    buy_signal_rsi = rsi < rsi_lower
    sell_signal_rsi = rsi > rsi_upper

    return buy_signal_rsi, sell_signal_rsi


def ema(data: pd.DataFrame, short_window: int, long_window: int) -> tuple:
    short_ema = ta.trend.EMAIndicator(data['Close'], window=short_window).ema_indicator()
    long_ema = ta.trend.EMAIndicator(data['Close'], window=long_window).ema_indicator()

    buy_signal_ema = short_ema > long_ema
    sell_signal_ema = short_ema < long_ema

    return buy_signal_ema, sell_signal_ema


def stochastic_oscillator(data: pd.DataFrame, k_window: int, d_window: int) -> tuple:
    stoch_indicator = ta.momentum.StochasticOscillator(data['High'], data['Low'], data['Close'], window=k_window, smooth_window=d_window)
    stoch_k = stoch_indicator.stoch()
    stoch_d = stoch_indicator.stoch_signal()

    buy_signal_stoch = (stoch_k < 20) & (stoch_d < 20) & (stoch_k > stoch_d)
    sell_signal_stoch = (stoch_k > 80) & (stoch_d > 80) & (stoch_k < stoch_d)

    return buy_signal_stoch, sell_signal_stoch

In [3]:
def backtest(data, trail) -> float:
    data = data.copy()

    # --- Parámetros del trail ---
    rsi_window = trail.suggest_int('rsi_window', 5, 50)
    rsi_lower = trail.suggest_int('rsi_lower', 5, 35)
    rsi_upper = trail.suggest_int('rsi_upper', 65, 95)

    short_window = trail.suggest_int('short_window', 5, 20)
    long_window = trail.suggest_int('long_window', 30, 100)

    k_window = trail.suggest_int('k_window', 5, 20)
    d_window = trail.suggest_int('d_window', 3, 10)

    stop_loss = trail.suggest_float('stop_loss', 0.01, 0.15)
    take_profit = trail.suggest_float('take_profit', 0.01, 0.15)
    n_shares = trail.suggest_int('n_shares', 50, 500)

    # --- Señales ---
    rsi_buy, rsi_sell = rsi(data, rsi_window, rsi_lower, rsi_upper)
    ema_buy, ema_sell = ema(data, short_window, long_window)
    stoch_buy, stoch_sell = stochastic_oscillator(data, k_window, d_window)

    historic = data.dropna()
    historic['buy_signal'] = (rsi_buy.astype(int) + ema_buy.astype(int) + stoch_buy.astype(int)) >= 2
    historic['sell_signal'] = (rsi_sell.astype(int) + ema_sell.astype(int) + stoch_sell.astype(int)) >= 2  

    # --- Variables de portafolio ---
    COM = 0.125 / 100
    SL = stop_loss
    TP = take_profit
    cash = 1_000_000

    active_long_positions: list[Operation] = []
    active_short_positions: list[Operation] = []
    portfolio_values = [cash]

    # --- Loop sobre histórico ---
    for i, row in historic.iterrows():  

        # --- Revisar LONGS abiertos ---
        for position in active_long_positions.copy():
            if row.Close >= position.take_profit or row.Close <= position.stop_loss:
                cash += row.Close * position.n_shares * (1 - COM)
                active_long_positions.remove(position)

        # --- Revisar SHORTS abiertos ---
        for position in active_short_positions.copy():
            if row.Close <= position.take_profit or row.Close >= position.stop_loss:
                cash -= row.Close * position.n_shares * (1 + COM)
                pnl = (position.price - row.Close) * position.n_shares 
                cash += pnl
                active_short_positions.remove(position)

        # --- Abrir LONG ---
        if row.buy_signal:
            if cash >= row.Close * n_shares * (1 + COM):
                cash -= row.Close * n_shares * (1 + COM)
                active_long_positions.append(
                    Operation(
                        time=row.Datetime,
                        price=row.Close,
                        stop_loss=row.Close * (1 - SL),
                        take_profit=row.Close * (1 + TP),
                        n_shares=n_shares,
                        type='LONG'
                    )
                )

        # --- Abrir SHORT ---
        if row.sell_signal:
            if cash >= row.Close * n_shares * (1 + COM):
                cash -= row.Close * n_shares * (1 + COM)
                active_short_positions.append(
                    Operation(
                        time=row.Datetime,
                        price=row.Close,
                        stop_loss=row.Close * (1 + SL),
                        take_profit=row.Close * (1 - TP),
                        n_shares=n_shares,
                        type='SHORT'
                    )
             )

        # --- Guardar valor del portafolio ---
        portfolio_values.append(
            get_portfolio_value(cash, active_long_positions, active_short_positions, row.Close, n_shares, COM)
        )
    

    for position in active_long_positions:
        cash += position.n_shares * row.Close * (1 - COM)

    for position in active_short_positions:
        cash -= position.n_shares * row.Close * (1 + COM)
        pnl = (position.price - row.Close) * position.n_shares
        cash += pnl

    active_long_positions = []
    active_short_positions = []
    portfolio_values.append(cash)

    # --- Calmar Ratio ---
    calmar_df = pd.DataFrame(portfolio_values, columns=['Portfolio Value'])
    calmar_val = calmar_ratio(calmar_df['Portfolio Value']) 

    return calmar_val


In [4]:
def backtest_with_params(data, params: dict) -> float:
    data = data.copy()

    # --- Señales ---
    rsi_buy, rsi_sell = rsi(data, params['rsi_window'], params['rsi_lower'], params['rsi_upper'])
    ema_buy, ema_sell = ema(data, params['short_window'], params['long_window'])
    stoch_buy, stoch_sell = stochastic_oscillator(data, params['k_window'], params['d_window'])

    historic = data.dropna()
    historic['buy_signal'] = (rsi_buy.astype(int) + ema_buy.astype(int) + stoch_buy.astype(int)) >= 2
    historic['sell_signal'] = (rsi_sell.astype(int) + ema_sell.astype(int) + stoch_sell.astype(int)) >= 2  

    # --- Variables de portafolio ---
    COM = 0.125 / 100
    SL = params['stop_loss']
    TP = params['take_profit']
    n_shares = params['n_shares']
    cash = 1_000_000

    active_long_positions: list[Operation] = []
    active_short_positions: list[Operation] = []
    portfolio_values = [cash]

    # --- Loop sobre histórico ---
    for i, row in historic.iterrows():  

        # --- Revisar LONGS abiertos ---
        for position in active_long_positions.copy():
            if row.Close >= position.take_profit or row.Close <= position.stop_loss:
                cash += row.Close * position.n_shares * (1 - COM)
                active_long_positions.remove(position)

        # --- Revisar SHORTS abiertos ---
        for position in active_short_positions.copy():
            if row.Close <= position.take_profit or row.Close >= position.stop_loss:
                cash -= row.Close * position.n_shares * (1 + COM)
                pnl = (position.price - row.Close) * position.n_shares 
                cash += pnl
                active_short_positions.remove(position)

        # --- Abrir LONG ---
        if row.buy_signal:
            if cash >= row.Close * n_shares * (1 + COM):
                cash -= row.Close * n_shares * (1 + COM)
                active_long_positions.append(
                    Operation(
                        time=row.Datetime,
                        price=row.Close,
                        stop_loss=row.Close * (1 - SL),
                        take_profit=row.Close * (1 + TP),
                        n_shares=n_shares,
                        type='LONG'
                    )
                )

        # --- Abrir SHORT ---
        if row.sell_signal:
            if cash >= row.Close * n_shares * (1 + COM):
                cash -= row.Close * n_shares * (1 + COM)
                active_short_positions.append(
                    Operation(
                        time=row.Datetime,
                        price=row.Close,
                        stop_loss=row.Close * (1 + SL),
                        take_profit=row.Close * (1 - TP),
                        n_shares=n_shares,
                        type='SHORT'
                    )
                )

        # --- Guardar valor del portafolio ---
        portfolio_values.append(
            get_portfolio_value(cash, active_long_positions, active_short_positions, row.Close, n_shares, COM)
        )
    
    # --- Cierre de posiciones abiertas ---
    for position in active_long_positions:
        cash += position.n_shares * row.Close * (1 - COM)

    for position in active_short_positions:
        cash -= position.n_shares * row.Close * (1 + COM)
        pnl = (position.price - row.Close) * position.n_shares
        cash += pnl

    portfolio_values.append(cash)

    # --- Calmar Ratio ---
    calmar_df = pd.DataFrame(portfolio_values, columns=['Portfolio Value'])
    calmar_val = calmar_ratio(calmar_df['Portfolio Value']) 

    return calmar_val
